In [1]:

import pandas as pd



In [2]:
!pip install psycopg2

In [3]:
# Load the CSV file to inspect
file_path = ("complete_data.csv")

# Read a few rows from the file
df = pd.read_csv(file_path)

# Display the first few rows
print("CSV Data Preview:")
print(df.head())

CSV Data Preview:
       Date  Index SA Redfin HPI MoM  Case Shiller Index MoM  period_duration  \
0  1/1/2012      59.9          0.00%                -0.00048               30   
1  1/1/2012      59.9          0.00%                -0.00048               30   
2  1/1/2012      59.9          0.00%                -0.00048               30   
3  1/1/2012      59.9          0.00%                -0.00048               30   
4  1/1/2012      59.9          0.00%                -0.00048               30   

  region_type  table_id is_seasonally_adjusted         region          state  \
0       state        23                      f       Oklahoma       Oklahoma   
1       state        10                      f  New Hampshire  New Hampshire   
2       state        42                      f       Virginia       Virginia   
3       state        47                      f       Michigan       Michigan   
4       state        12                      f     New Jersey     New Jersey   

   ... price_d

In [4]:
# First, let's check what columns are in the DataFrame
print("Available columns in DataFrame:", df.columns.tolist())

Available columns in DataFrame: ['Date', 'Index SA', 'Redfin HPI MoM', 'Case Shiller Index MoM', 'period_duration', 'region_type', 'table_id', 'is_seasonally_adjusted', 'region', 'state', 'property_type', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_t

In [5]:
# First, check if the date column exists in any of these common formats
possible_date_cols = ['Date', 'date', 'DATE', 'datetime', 'Datetime', 'time']

# Find the date column if it exists
date_col = None
for col in possible_date_cols:
    if col in df.columns:
        date_col = col
        break

# Only proceed with date conversion if we found a date column
if date_col:
    # Convert 'Date' to proper format
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Standardize the column name to 'date'
    if date_col != 'date':
        df = df.rename(columns={date_col: 'date'})
else:
    print("Warning: No date column found in the DataFrame")

# Rest of your code remains the same
# Convert percentage columns to numeric (remove '%' and divide by 100)
percent_columns = ['Redfin HPI MoM', 'Case Shiller Index MoM', 'median_sale_price_mom', 
                   'median_sale_price_yoy', 'median_list_price_mom', 'median_list_price_yoy',
                   'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf_mom', 'median_list_ppsf_yoy',
                   'homes_sold_mom', 'homes_sold_yoy', 'pending_sales_mom', 'pending_sales_yoy',
                   'new_listings_mom', 'new_listings_yoy', 'inventory_mom', 'inventory_yoy',
                   'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom_mom', 'median_dom_yoy',
                   'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list_mom', 'sold_above_list_yoy',
                   'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_weeks_mom', 'off_market_in_two_weeks_yoy']

for col in percent_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace('%', '').astype(float) / 100

# Convert boolean columns
bool_columns = ['is_seasonally_adjusted', 'price_drops_is_blank', 'price_drops_mom_is_blank', 'price_drops_yoy_is_blank']
for col in bool_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).map({'t': True, 'f': False, 'True': True, 'False': False})

# Standardize column names to match SQL table
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Save cleaned data to a new CSV file
cleaned_file_path = "cleaned_completed_data.csv"
df.to_csv(cleaned_file_path, index=False)

# Display cleaned data for review
print("Cleaned CSV Data:")
print(df.head())

# Return the cleaned file path for PostgreSQL loading
cleaned_file_path

Cleaned CSV Data:
        date  index_sa  redfin_hpi_mom  case_shiller_index_mom  \
0 2012-01-01      59.9             0.0               -0.000005   
1 2012-01-01      59.9             0.0               -0.000005   
2 2012-01-01      59.9             0.0               -0.000005   
3 2012-01-01      59.9             0.0               -0.000005   
4 2012-01-01      59.9             0.0               -0.000005   

   period_duration region_type  table_id  is_seasonally_adjusted  \
0               30       state        23                   False   
1               30       state        10                   False   
2               30       state        42                   False   
3               30       state        47                   False   
4               30       state        12                   False   

          region          state  ... price_drops  price_drops_mom  \
0       Oklahoma       Oklahoma  ...    0.000000         0.000000   
1  New Hampshire  New Hampshire  ...  

'cleaned_completed_data.csv'

In [6]:
print("Cleaned CSV Data:")
print(df.head())

Cleaned CSV Data:
        date  index_sa  redfin_hpi_mom  case_shiller_index_mom  \
0 2012-01-01      59.9             0.0               -0.000005   
1 2012-01-01      59.9             0.0               -0.000005   
2 2012-01-01      59.9             0.0               -0.000005   
3 2012-01-01      59.9             0.0               -0.000005   
4 2012-01-01      59.9             0.0               -0.000005   

   period_duration region_type  table_id  is_seasonally_adjusted  \
0               30       state        23                   False   
1               30       state        10                   False   
2               30       state        42                   False   
3               30       state        47                   False   
4               30       state        12                   False   

          region          state  ... price_drops  price_drops_mom  \
0       Oklahoma       Oklahoma  ...    0.000000         0.000000   
1  New Hampshire  New Hampshire  ...  

In [7]:
df.head()

,date,index_sa,redfin_hpi_mom,case_shiller_index_mom,period_duration,region_type,table_id,is_seasonally_adjusted,region,state,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,30_year_%,price_drops_is_blank,price_drops_mom_is_blank,price_drops_yoy_is_blank
0,2012-01-01,59.9,0.0,-0.000005,30,state,23,False,Oklahoma,Oklahoma,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.915,True,True,True
1,2012-01-01,59.9,0.0,-0.000005,30,state,10,False,New Hampshire,New Hampshire,...,0.000000,0.000000,0.0,0.007093,0.000006,-0.000002,3.915,True,True,True
2,2012-01-01,59.9,0.0,-0.000005,30,state,42,False,Virginia,Virginia,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.915,True,True,True
3,2012-01-01,59.9,0.0,-0.000005,30,state,47,False,Michigan,Michigan,...,0.000000,0.000000,0.0,0.013045,-0.000005,0.000007,3.915,True,True,True
4,2012-01-01,59.9,0.0,-0.000005,30,state,12,False,New Jersey,New Jersey,...,0.000549,0.000005,0.0,0.007897,0.000018,0.000061,3.915,False,False,True


In [8]:
!pip install psycopg2

In [15]:
import os
os.environ["PG_PASSWORD"] = "Addis@11" 

In [17]:
print(os.getenv("PG_PASSWORD"))



Addis@11


In [19]:
import pandas as pd
import psycopg2 
import os

# Database connection settings

db_params = {
    "dbname": "project_4",
    "user": "postgres",
    "password": os.getenv("PG_PASSWORD"),
    "host": "localhost"
    
}

# Connect to PostgreSQL
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [21]:
sql_query = """
    INSERT INTO housing_data (
        date, index_sa, redfin_hpi_mom, case_shiller_index_mom, 
        period_duration, region_type, table_id, is_seasonally_adjusted, 
        region, state, property_type, median_sale_price, 
        median_sale_price_mom, median_sale_price_yoy, median_list_price, 
        median_list_price_mom, median_list_price_yoy, median_ppsf, 
        median_ppsf_mom, median_ppsf_yoy, median_list_ppsf, 
        median_list_ppsf_mom, median_list_ppsf_yoy, homes_sold, 
        homes_sold_mom, homes_sold_yoy, pending_sales, pending_sales_mom, 
        pending_sales_yoy, new_listings, new_listings_mom, new_listings_yoy, 
        inventory, inventory_mom, inventory_yoy, months_of_supply, 
        months_of_supply_mom, months_of_supply_yoy, median_dom, 
        median_dom_mom, median_dom_yoy, avg_sale_to_list, 
        avg_sale_to_list_mom, avg_sale_to_list_yoy, sold_above_list, 
        sold_above_list_mom, sold_above_list_yoy, price_drops, 
        price_drops_mom, price_drops_yoy, off_market_in_two_weeks, 
        off_market_in_two_weeks_mom, off_market_in_two_weeks_yoy, 
        year_30_rate, price_drops_is_blank, price_drops_mom_is_blank, 
        price_drops_yoy_is_blank
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

print(f"Number of DataFrame columns: {len(df.columns)}")
print(f"Expected placeholders: {sql_query.count('%s')}")  

conn.commit()
print("Data Successfully Inserted")   


Number of DataFrame columns: 57
Expected placeholders: 55
Data Successfully Inserted


In [23]:
conn.commit()
print("Data Successfully Inserted") 
# Fetch and print 5 rows before closing the connection
cur.execute("SELECT * FROM housing_data LIMIT 5")  
rows = cur.fetchall()
for row in rows:
    print(row)

#close the connection
conn.close()

Data Successfully Inserted
(datetime.date(2012, 1, 1), 59.9, 0.0, -4.8e-06, 30, 'state', 23, False, 'Oklahoma', 'Oklahoma', 'Townhouse', 87200, -0.003837864839791622, -0.005146831530139105, 130000.0, 0.000842368640533777, -0.000363232023721274, 18.0, -0.003165117031461115, -0.00292713453291632, 76.0, 0.000797117067576551, -0.00026518374458085297, 9, -0.001818181818181817, 0.0, 9.0, -0.001818181818181817, 0.0, 6.0, 0.01, -0.001428571428571429, 17.0, -0.001052631578947368, -0.003928571428571429, 1.9, 0.001999999999999999, -0.012000000000000002, 14.0, -0.05, -0.94, 0.9302420431048024, -0.000353365018605132, -0.000227314547174923, 0.0, -0.001818181818181818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.915, True, True, True)
(datetime.date(2012, 1, 1), 59.9, 0.0, -4.8e-06, 30, 'state', 10, False, 'New Hampshire', 'New Hampshire', 'All Residential', 184000, -0.000173976610967446, -0.000286861258824177, 199300.0, 0.001313254597175108, 0.000343174775093182, 111.0, 0.00030914251922934804, 0.0001801406